# Project

Welcome to the group project! The project is based on the [ACM RecSys 2021 Challenge](https://recsys-twitter.com/).

- Detailed information about the task, submission and grading can be found in a [dedicates site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1217340).
- Information about the dataset structure [on this site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1218810).

In [84]:
import pandas as pd
import numpy as np
team_name = ""
team_members = ""

In [85]:
print(team_name)
print(team_members)

In [86]:
import os
import re
import csv
import datetime

# from model import reply_pred_model, retweet_pred_model, quote_pred_model, fav_pred_model 

all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply", "retweet", "quote", "like"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

In [87]:
########## TO CHANGE #################

path_to_data = './data/project/training/' 
val_path_to_data = './data/project/validation/'

dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week
expanded_path = os.path.expanduser(path_to_data)
part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
part_files = sorted(part_files, key = lambda x:x[-5:])
part_files

['./data/project/training/one_hour.csv']

Features to consider:
- Photo?
- Video?
- Link?
- Tweet type:  Retweet, Quote or Toplevel
- Follower count of engaged with user (log10)
- Following count of engaged with user (log10)
- Is verified? of engaged with user
- rel nr of engagements


In [88]:
feature_names = [
    "photo",
    "video",
    "link",
    "retweet",
    "quote",
    "toplevel",
    "follower_count",
    "following_count",
    "verified",
    "rel_engagement"
]
target_names = [
    "rel_replies",
    "rel_retweets",
    "rel_quotes",
    "rel_likes"
]

feature_to_col = dict(zip(feature_names, range(len(feature_names))))
print(feature_to_col)
target_to_col = dict(zip(target_names, range(len(target_names))))
print(target_to_col)

{'photo': 0, 'video': 1, 'link': 2, 'retweet': 3, 'quote': 4, 'toplevel': 5, 'follower_count': 6, 'following_count': 7, 'verified': 8, 'rel_engagement': 9}
{'rel_replies': 0, 'rel_retweets': 1, 'rel_quotes': 2, 'rel_likes': 3}


In [89]:
tweet_ids = {}
i = 0
total_entries = 0

with open(part_files[0], 'r') as f:
    linereader = csv.reader(f, delimiter='\x01')
    for row in linereader:
        tweet_id = row[all_features_to_idx["tweet_id"]]
        if tweet_id not in tweet_ids:
            tweet_ids[tweet_id] = i
            i += 1
        total_entries += 1
        
total_entries

891747

In [90]:
user_ids = {}
i = 0

with open(part_files[0], 'r') as f:
    linereader = csv.reader(f, delimiter='\x01')
    for row in linereader:
        user_id = row[all_features_to_idx["engaging_user_id"]]
        if user_id not in user_ids:
            user_ids[user_id] = i
            i += 1

In [91]:
n_tweets = len(tweet_ids)
n_tweets

462701

In [92]:
tweet_features = np.zeros((len(tweet_ids), len(feature_names)), dtype=np.float32)
tweet_targets = np.zeros((len(tweet_ids), len(target_names)), dtype=np.float32)
counts = np.zeros((len(tweet_ids),), dtype=np.float32)

tweet_timestamps = np.zeros((len(tweet_ids)), dtype=np.int)
with open(part_files[0], 'r') as f:
    linereader = csv.reader(f, delimiter='\x01')
    j = 0
    for row in linereader:
        tweet_id = row[all_features_to_idx["tweet_id"]]
        index = tweet_ids[tweet_id]
        
        timestamp = int(row[all_features_to_idx["tweet_timestamp"]])
        if tweet_timestamps[index] != 0:
            assert timestamp == tweet_timestamps[index] # catch inconsistencies
        else:
            tweet_timestamps[index] = timestamp
            
            media = row[all_features_to_idx["present_media"]]
            if "Photo" in media:
                tweet_features[index, feature_to_col["photo"]] = 1
            
            if "Video" in media or "GIF" in media:
                tweet_features[index, feature_to_col["video"]] = 1
            
            if row[all_features_to_idx["present_links"]] != "":
                tweet_features[index, feature_to_col["link"]] = 1
                
            tweet_type = row[all_features_to_idx["tweet_type"]]
            if tweet_type == "TopLevel":
                tweet_features[index, feature_to_col["toplevel"]] = 1
            elif tweet_type == "Quote":
                tweet_features[index, feature_to_col["quote"]] = 1
            elif tweet_type == "Retweet":
                tweet_features[index, feature_to_col["retweet"]] = 1
                
            tweet_features[index, feature_to_col["follower_count"]] = int(row[all_features_to_idx["engaged_with_user_following_count"]])
            tweet_features[index, feature_to_col["following_count"]] = int(row[all_features_to_idx["engaged_with_user_follower_count"]])
            tweet_features[index, feature_to_col["verified"]] = bool(row[all_features_to_idx["engaged_with_user_is_verified"]])

        
        if row[all_features_to_idx["reply"]]:
            tweet_targets[index, target_to_col["rel_replies"]] += 1

        if row[all_features_to_idx["retweet"]]:
            tweet_targets[index, target_to_col["rel_retweets"]] += 1
            
        if row[all_features_to_idx["quote"]]:
            tweet_targets[index, target_to_col["rel_quotes"]] += 1
            
        if row[all_features_to_idx["like"]]:
            tweet_targets[index, target_to_col["rel_likes"]] += 1
            
        
        counts[index] += 1
            
        j += 1
        if j % 10000 == 0:
            print(f"{j}/{total_entries}", end="\r")
            

print(f"{j}/{total_entries}", end="\r")
            
tweet_features[:, feature_to_col["rel_engagement"]] = (
    tweet_targets[:, target_to_col["rel_replies"]] +
    tweet_targets[:, target_to_col["rel_retweets"]] +
    tweet_targets[:, target_to_col["rel_quotes"]] +
    tweet_targets[:, target_to_col["rel_likes"]])
        
tweet_features[:,feature_to_col["follower_count"]] = np.log10(tweet_features[:,feature_to_col["follower_count"]] + 1)
tweet_features[:,feature_to_col["following_count"]] = np.log10(tweet_features[:,feature_to_col["following_count"]] + 1)

tweet_targets[:, target_to_col["rel_replies"]] /= counts
tweet_targets[:, target_to_col["rel_retweets"]] /= counts
tweet_targets[:, target_to_col["rel_quotes"]] /= counts
tweet_targets[:, target_to_col["rel_likes"]] /= counts
tweet_features[:, feature_to_col["rel_engagement"]] /= counts

In [93]:
scaling = tweet_features.max(axis=0)
for i, (c, s) in enumerate(zip(feature_names, scaling)):
    print(i, c, s)

0 photo 1.0
1 video 1.0
2 link 1.0
3 retweet 1.0
4 quote 1.0
5 toplevel 1.0
6 follower_count 6.167881
7 following_count 7.7219787
8 verified 1.0
9 rel_engagement 4.0


In [94]:
tweet_features /= np.linalg.norm(tweet_features, axis=1).reshape(-1,1)

In [95]:
np.linalg.norm(tweet_features, axis=1)

array([1.        , 0.99999994, 1.        , ..., 1.        , 0.99999994,
       1.        ], dtype=float32)

In [96]:
tweet_features.dot(tweet_features[0,:])

array([1.        , 0.93883604, 0.9520716 , ..., 0.91096187, 0.9776502 ,
       0.9282817 ], dtype=float32)

In [97]:
similarities = tweet_features.dot(tweet_features[0,:])

In [98]:
topk = 20
amaxes = np.argpartition(similarities, -topk)[-topk:] # linear time
amaxes

array([394474, 214827, 450887,   3014,  29126, 319318, 169587,  54117,
       184192,  46031, 266746,   5331, 347052, 335048,  77966, 355213,
       341221, 240096,  50880,      0], dtype=int64)

In [99]:
similarities[amaxes]

array([0.9999946 , 0.99999475, 0.99999464, 0.9999954 , 0.9999958 ,
       0.999996  , 0.99999547, 0.9999962 , 0.99999696, 0.9999996 ,
       0.99999934, 0.9999971 , 0.99999624, 0.9999989 , 0.9999992 ,
       0.9999991 , 0.9999999 , 0.9999999 , 0.99999774, 1.        ],
      dtype=float32)

In [100]:
weights = similarities[amaxes]
weights /= np.abs(weights).sum()
weights

array([0.04999986, 0.04999987, 0.04999987, 0.0499999 , 0.04999993,
       0.04999993, 0.04999991, 0.04999994, 0.04999998, 0.05000011,
       0.0500001 , 0.04999999, 0.04999994, 0.05000008, 0.05000009,
       0.05000009, 0.05000013, 0.05000013, 0.05000002, 0.05000013],
      dtype=float32)

In [101]:
tweet_targets[amaxes,:]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [102]:
weights.reshape((-1,1))

array([[0.04999986],
       [0.04999987],
       [0.04999987],
       [0.0499999 ],
       [0.04999993],
       [0.04999993],
       [0.04999991],
       [0.04999994],
       [0.04999998],
       [0.05000011],
       [0.0500001 ],
       [0.04999999],
       [0.04999994],
       [0.05000008],
       [0.05000009],
       [0.05000009],
       [0.05000013],
       [0.05000013],
       [0.05000002],
       [0.05000013]], dtype=float32)

In [103]:
target_prediction = np.sum(tweet_targets[amaxes,:] * weights.reshape((-1,1)), axis=0)
target_prediction

array([0.09999999, 0.14999996, 0.        , 0.75000006], dtype=float32)

In [104]:
target_names

['rel_replies', 'rel_retweets', 'rel_quotes', 'rel_likes']

In [105]:
reply_pred, retweet_pred, quote_pred, fav_pred = target_prediction

In [106]:
def parse_input_line(row):
    tweet_id = row[all_features_to_idx['tweet_id']]
    user_id = row[all_features_to_idx['engaging_user_id']]
    
    input_feats = np.zeros((tweet_features.shape[1],),dtype=np.float32)
    
    media = row[all_features_to_idx["present_media"]]
    if "Photo" in media:
        input_feats[feature_to_col["photo"]] = 1

    if "Video" in media or "GIF" in media:
        input_feats[feature_to_col["video"]] = 1

    if row[all_features_to_idx["present_links"]] != "":
        input_feats[feature_to_col["link"]] = 1

    tweet_type = row[all_features_to_idx["tweet_type"]]
    if tweet_type == "TopLevel":
        input_feats[feature_to_col["toplevel"]] = 1
    elif tweet_type == "Quote":
        input_feats[feature_to_col["quote"]] = 1
    elif tweet_type == "Retweet":
        input_feats[feature_to_col["retweet"]] = 1

    input_feats[feature_to_col["follower_count"]] = int(row[all_features_to_idx["engaged_with_user_following_count"]])
    input_feats[feature_to_col["following_count"]] = int(row[all_features_to_idx["engaged_with_user_follower_count"]])
    input_feats[feature_to_col["verified"]] = bool(row[all_features_to_idx["engaged_with_user_is_verified"]])
    
    input_feats /= np.linalg.norm(input_feats)
            
    tweet_timestamp = row[all_features_to_idx['tweet_timestamp']]
    
    return tweet_id, user_id, input_feats, tweet_timestamp


def evaluate_test_set(path, dataset_type):
    expanded_path = os.path.expanduser(path)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for i, row in enumerate(linereader):
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row) 
                    
                    if tweet_id in tweet_ids:
                        # allows more features, like rel_engagement, which are otherwise 0
                        features = tweet_features[tweet_ids[tweet_id], :] 
                    
                    similarities = tweet_features.dot(features)
                    
                    topk = 20
                    amaxes = np.argpartition(similarities, -topk)[-topk:] # linear time
                    
                    weights = similarities[amaxes]
                    weights /= np.abs(weights).sum()

                    target_prediction = np.sum(tweet_targets[amaxes,:] * weights.reshape((-1,1)), axis=0)
                    
                    reply_pred, retweet_pred, quote_pred, fav_pred = target_prediction

                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')

                    if i % 1000 == 0:
                        print(i, end="\r")
                    i += 1
                
                print("Done!", end="\r")


In [107]:
%%time
evaluate_test_set(val_path_to_data, 'one_hour')

KeyboardInterrupt: 

In [117]:
def write_ground_truth(path, dataset_type):
    expanded_path = os.path.expanduser(path)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('gt.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id = row[all_features_to_idx['tweet_id']]
                    user_id = row[all_features_to_idx['engaging_user_id']]
                    
                    reply_pred = int(row[all_features_to_idx['reply']] != "") 
                    retweet_pred = int(row[all_features_to_idx['retweet']] != "")
                    quote_pred = int(row[all_features_to_idx['quote']] != "") 
                    fav_pred = int(row[all_features_to_idx['like']] != "") 
                    
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')

In [109]:
write_ground_truth(val_path_to_data, 'one_hour')

In [118]:
from sklearn.metrics import average_precision_score, log_loss

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])

    ctr = positive/float(len(gt))

    return ctr

def compute_rce(pred, gt):

    cross_entropy = log_loss(gt, pred)

    data_ctr = calculate_ctr(gt)

    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])

    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0


#### zeroR
rce: -621.8494688006682  
average_precision: 0.02245872690425116  
accuracy: 0.9775412730957488  

#### item item
rce: -233.43933830426766  
average_precision: 0.018494913120772966  
accuracy: 0.22894864714317884

## Fairness

In [119]:
def read_predictions_fairness(path):
    pred = pd.read_csv(path, header=None, names=['tweet_id', 'user_id', 'reply', 'retweet', 'quote', 'like'])
    return pred

def read_groundtruth_fairness(path, col):
    pred = pd.read_csv(path, header=None, names=col)
    return pred

In [120]:
def parse_line(row):
    tweet_id = row[all_features_to_idx['tweet_id']]
    user_id = row[all_features_to_idx['engaging_user_id']]
    
    input_feats = np.zeros((tweet_features.shape[1],),dtype=np.float32)
    
    follower_count= int(row[all_features_to_idx["engaged_with_user_follower_count"]])
    following_count = int(row[all_features_to_idx["engaged_with_user_following_count"]])
    verified = bool(row[all_features_to_idx["engaged_with_user_is_verified"]])
    
    return tweet_id, user_id, follower_count, following_count, verified

In [154]:
def get_tweet_ids(path):
    tweet_ids = {}
    i = 0
    total_entries = 0

    with open(path, 'r') as f:
        linereader = csv.reader(f, delimiter='\x01')
        for row in linereader:
            tweet_id = row[all_features_to_idx["tweet_id"]]
            if tweet_id not in tweet_ids:
                tweet_ids[tweet_id] = i
                i += 1
            total_entries += 1

    return tweet_ids

def get_user_ids(path):
    user_ids = {}
    i = 0

    with open(path, 'r') as f:
        linereader = csv.reader(f, delimiter='\x01')
        for row in linereader:
            user_id = row[all_features_to_idx["engaging_user_id"]]
            if user_id not in user_ids:
                user_ids[user_id] = i
                i += 1
    return user_ids

In [156]:
def tweets_data(path, dataset_type):
    expanded_path = os.path.expanduser(path)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
    
    tweet_groups = pd.DataFrame(columns=['tweet_id', 'user_id', 'follower_count', 'following_count', 'verified'])
    
    for file in part_files:
        with open(file, 'r') as f:
            tweet_ids = get_tweet_ids(file)
            user_ids = get_user_ids(file)
            linereader = csv.reader(f, delimiter='\x01')
            last_timestamp = None
            for i, row in enumerate(linereader):
                tweet_id, user_id, follower_count, following_count, verified = parse_line(row) 
                tweet_id_int = tweet_ids[tweet_id]
                user_id_int = user_ids[user_id]
                dic = {'tweet_id':tweet_id_int, 'user_id':user_id_int,\
                       'follower_count':follower_count, 'following_count':following_count, 'verified':verified}
                tweet_groups = tweet_groups.append(dic, ignore_index=True)
    return tweet_ids, user_ids, tweet_groups

tweet_ids, user_ids, tweet_groups = tweets_data(val_path_to_data, 'one_hour')

In [158]:
tweet_groups

,tweet_id,user_id,follower_count,following_count,verified
0,0,0,54184,731,True
1,1,1,398780,15,True
2,1,2,398780,15,True
3,2,3,53517,411,True
4,3,4,14083,316,True
...,...,...,...,...,...
84372,58285,82821,3132,4409,True
84373,58286,82822,305,88,True
84374,58287,82823,882,637,True
84375,58288,82824,92036,1813,True


In [210]:
def group_by_followers(df):
    data = df.copy()
    data = data.sort_values(by='follower_count', ascending=False)
    data['group'] = np.zeros((len(data)), dtype=np.int32)
    
    for i in range(0,round(len(data)/5)):
        data.loc[i, 'group'] = 0
        
    for i in range(round(len(data)/5), 2*round(len(data)/5)):
        data.loc[i, 'group'] = 1
        
    for i in range(2*round(len(data)/5), 3*round(len(data)/5)):
        data.loc[i, 'group'] = 2
        
    for i in range(3*round(len(data)/5), 4*round(len(data)/5)):
        data.loc[i, 'group'] = 3
        
    for i in range(4*round(len(data)/5), len(data)):
        data.loc[i, 'group'] = 4
    
    return data

groups = group_by_followers(tweet_groups)



In [211]:
predictions = read_predictions_fairness("results.csv") # will return data in the form (tweet_id, user_id, prediction)

In [225]:
from sklearn.metrics import average_precision_score, log_loss
  
ground_truth = read_groundtruth_fairness("gt.csv") # will return data in the form (tweet_id, user_id, labed (1 or 0))
predictions = read_predictions_fairness("results.csv") # will return data in the form (tweet_id, user_id, prediction)

predictions['tweet_id'] = predictions['tweet_id'].map(tweet_ids)
predictions['user_id'] = predictions['user_id'].map(user_ids)
ground_truth['tweet_id'] = ground_truth['tweet_id'].map(tweet_ids)
ground_truth['user_id'] = ground_truth['user_id'].map(user_ids)

predictions = pd.merge(predictions, groups[['user_id', 'group']], how='left', on = 'user_id')
ground_truth = pd.merge(ground_truth, groups[['user_id', 'group']], how='left', on = 'user_id')





IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [223]:
ground_truth

,tweet_id,user_id,reply,retweet,quote,like,group
0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0
2,1,2,0,0,0,0,0
3,2,3,0,0,0,0,0
4,3,4,0,0,0,1,0
...,...,...,...,...,...,...,...
87612,58285,82821,0,0,0,0,4
87613,58286,82822,0,0,0,0,4
87614,58287,82823,0,0,0,1,4
87615,58288,82824,0,0,0,0,4


In [222]:
# splitting for groups (this is assuming ground_truth is a csv again)
# also assuming that predictions has class membership. We should sort this out in the least painful way
# predictions has schema (tweet_id, user_id, , group, prediction)
rce = {}
average_precision = {}
for i in range(5):
    group_predictions = [p for p in predictions.values if p[6] == i]
    group_ground_truth = [p for p in ground_truth.values if p[6] == i]
    rce[i] = compute_rce(group_predictions, group_ground_truth[:len(group_predictions)])
    average_precision[i] = average_precision_score(ground_truth, predictions)

rce = np.mean(rce.values)
average_precision = np.mean(average_precision.values)


ValueError: Unknown label type: ([array([0, 0, 0, 0, 0, 0, 0], dtype=object), array([1, 1, 0, 0, 0, 0, 0], dtype=object), array([1, 2, 0, 0, 0, 0, 0], dtype=object), array([2, 3, 0, 0, 0, 0, 0], dtype=object), array([3, 4, 0, 0, 0, 1, 0], dtype=object), array([4, 5, 0, 0, 0, 0, 0], dtype=object), array([5, 6, 0, 0, 0, 1, 0], dtype=object), array([6, 7, 0, 0, 0, 1, 0], dtype=object), array([6, 8, 0, 0, 0, 1, 0], dtype=object), array([6, 9, 0, 0, 0, 1, 0], dtype=object), array([7, 10, 0, 0, 0, 0, 0], dtype=object), array([8, 11, 0, 0, 0, 0, 0], dtype=object), array([9, 12, 0, 0, 0, 1, 0], dtype=object), array([10, 13, 0, 0, 0, 1, 0], dtype=object), array([10, 14, 0, 0, 0, 1, 0], dtype=object), array([11, 15, 0, 0, 0, 1, 0], dtype=object), array([12, 16, 0, 0, 0, 1, 0], dtype=object), array([12, 17, 0, 0, 0, 1, 0], dtype=object), array([12, 18, 0, 0, 0, 0, 0], dtype=object), array([12, 19, 0, 0, 0, 0, 0], dtype=object), array([13, 20, 0, 0, 0, 1, 0], dtype=object), array([14, 21, 0, 0, 0,

In [221]:
ground_truth


,tweet_id,user_id,reply,retweet,quote,like,group
0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0
2,1,2,0,0,0,0,0
3,2,3,0,0,0,0,0
4,3,4,0,0,0,1,0
...,...,...,...,...,...,...,...
87612,58285,82821,0,0,0,0,4
87613,58286,82822,0,0,0,0,4
87614,58287,82823,0,0,0,1,4
87615,58288,82824,0,0,0,0,4


## Data Analysis

In [ ]:
all_features_to_idx["tweet_id"], all_features_to_idx["engaged_with_user_id"], all_features_to_idx["engaging_user_id"]

In [ ]:
ids_df = pd.read_csv(part_files[0], delimiter='\x01', header=None, usecols=[2, 9, 14], low_memory=True)
ids_df.columns = ["tweet_id", "engaged_with_user_id", "engaging_user_id"]

In [ ]:
ids_df.memory_usage(deep=True).sum() / 1024**2

In [ ]:
ids_df

In [ ]:
count = ids_df["tweet_id"].unique().shape[0]
count, ids_df.shape[0],  count/ ids_df.shape[0] * 100

In [ ]:
count = ids_df["engaged_with_user_id"].unique().shape[0]
count, ids_df.shape[0], count/ ids_df.shape[0] * 100

In [ ]:
count = ids_df["engaging_user_id"].unique().shape[0]
count, ids_df.shape[0], count/ ids_df.shape[0] * 100

In [ ]:
count = np.unique(np.hstack((ids_df["engaged_with_user_id"].unique(), ids_df["engaging_user_id"].unique()))).shape[0]
count

In [ ]:
tweet_counts = ids_df["tweet_id"].value_counts()
tweet_counts[:20]

In [ ]:
for i in range(1,10):
    perc = tweet_counts[tweet_counts >= i].count() / tweet_counts.shape[0]
    print(i, perc)

In [ ]:
user_counts = ids_df["engaging_user_id"].value_counts()
user_counts[:20]

In [ ]:
for i in range(1,10):
    perc = user_counts[user_counts >= i].count() / user_counts.shape[0]
    print(i, perc)

In [ ]:
val_path_to_data = '~/shared/data/project/validation/'
val_expanded_path = os.path.expanduser(val_path_to_data)
val_part_files = [os.path.join(val_expanded_path, f) for f in os.listdir(val_expanded_path) if dataset_type in f]
val_part_files = sorted(val_part_files, key = lambda x:x[-5:])
val_part_files

In [ ]:
val_data = pd.read_csv(val_part_files[0], delimiter='\x01', header=None, usecols=[2, 9, 14])
val_data.columns = ["tweet_id", "engaged_with_user_id", "engaging_user_id"]
val_data

In [ ]:
len(set(val_data["engaging_user_id"].unique()).difference(set(user_counts.index.values))) / val_data.shape[0] * 100

In [ ]:
len(set(val_data["tweet_id"].unique()).difference(set(tweet_counts.index.values))) / val_data.shape[0] * 100

In [ ]:
# hidden


In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit